# Cuaderno para cargar metadatos 

Este cuaderno toma un directorio de MinIO con estructura de datos abiertos y crea la definición para Hive-Metastore para cada una de las tablas

## Librerias

In [ ]:
from minio import Minio
import pandas as pd
from io import StringIO
from io import BytesIO

## Definicion de coneccion a MinIO

In [ ]:
client = Minio(
        "minio:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    )

## Funciones de ayuda

In [ ]:
# Crea la definicion de la tabla en el directorio dado utilizando el diccionario de datos
def create_hive_table(client, bucket, directory_object_name):
    table_name = get_table_name(directory_object_name)
    col_def = get_col_def(client, bucket, directory_object_name)
    data_location = directory_object_name+"conjunto_de_datos/"
    # TODO: revisar si la tabla ya existe 
    table_def = """
        CREATE EXTERNAL TABLE {} ({})
        ROW FORMAT DELIMITED
        FIELDS TERMINATED BY ','
        LINES TERMINATED BY '\\n'
        LOCATION 's3a://{}/{}'
        TBLPROPERTIES ("skip.header.line.count"="1");
        """.format(table_name,col_def,bucket,data_location)
    return table_def

# Funcion para establecer el nombre de la tabla, en una version peeliminar es el nombre del directorio, pero se puede refinar para lograr algo mas conciso 
def get_table_name(directory_object_name):
    return directory_object_name.split("/")[-2]

# Fucion que crea la definicion de las variables, nombre de la variable y tipo 
def get_col_def (client, bucket, directory_object_name):
    data_dictionary = get_data_dictionary(client, bucket, directory_object_name)
    # TODO: agregar la parte donde podemos cambiar los nombres 
    # TODO: revisar si este ese el roden real de las columans de la tabla 
    names = [x.upper() for x in data_dictionary["Variable"]]
    types = [get_type(x) for x in data_dictionary["Tipo"]]
    return ", ".join(["{} {}".format(x,y) for x,y in zip(names,types)])
    
# Obtiene el diccionario de datos del formato de datos abiertos para definir las varibles
def get_data_dictionary(client, bucket, directory_object_name):
    dic_location = [obj.object_name for obj in client.list_objects(bucket, directory_object_name+"diccionario_de_datos/")]    
    try:
        response = client.get_object(bucket, dic_location[0])
        s = str(response.read(),'latin-1')
    finally:
        response.close()
        response.release_conn()
    df = pd.read_csv(StringIO(s),names=["id","Variable","Descripcion","Tipo","valor","etiqueta_rango"], index_col=False)
    valid_entries = ~pd.to_numeric(df["id"],errors='coerce',downcast='integer').isna()
    return df[valid_entries]

# Convierte el tipo de columna del estilo datos abiertos al estilo SQL
def get_type(entrada):
    # TODO: Investigar los tipos de HIve y ver cual se justa mejor
    # TODO: agregar lognitudes 
    lookup = {"C":"STRING","N":"DECIMAL"}
    for key in lookup:
        if entrada.startswith(key):
            return lookup[key]
    raise Exception('Variable del tipo "{}" no encontrado'.format(entrada))

# atraviesa todos los sub-directorios para crear un a tabla para cada uno
def create_dataset_tables(client,bucket,data_set):
    definitions = [create_hive_table(client, bucket, obj.object_name) for obj in client.list_objects(bucket, data_set) if obj.is_dir]
    return "\n".join(definitions)

# ToDo: ejecutar directamente en el Hive-Metastore y revisar si la definición ha sido exitosa 
    

## Ejemplo de ejecucion

En este ejemplo se toman los datos cargados en el cuaderno CargaObjetos.ipynb para crear las definiciones de tablas.

Es necesario copiar la salida de las siguientes celdas y pegarla en el archivo 'services\hivemetastore\create-table.hql'
y ejecutar el comando 
```
docker-compose exec hive-metastore beeline -u jdbc:hive2:// -f /tmp/create-table.hql
```
con esto se definiran los datos de las tablas. 


**Nota:** En los datos a descargar hay un problema con el archivo *sopresa* en la linea *desconocido* es necesario poner comillas para que el csv se detecte de manera correcta. 
Esto se debe hacer antes de cargarlo a MinIO, si apenas encontraste el error solo es necesario modificarlo y sobreescribir el archivo en MinIO

In [ ]:
print(create_dataset_tables(client,"hive","warehouse/conjunto_de_datos_enigh_2018_ns_csv/"))

In [ ]:
print(create_dataset_tables(client,"hive","warehouse/conjunto_de_datos_enigh2016_nueva_serie_csv/"))

In [ ]:
print(create_dataset_tables(client,"hive","warehouse/enigh_ncv_2014_csv/"))